In [1]:
import numpy as np
import seaborn as sns
import mdtraj as md
import matplotlib.pyplot as plt
import nglview as nv

# https://biopython.org/docs/1.74/api/Bio.SVDSuperimposer.html
from Bio.SVDSuperimposer import SVDSuperimposer

from numpy import array, dot, set_printoptions

# link to openmm manual for how to set up a minimizer
# http://docs.openmm.org/7.2.0/userguide/application.html

In [2]:
def get_rot_and_trans(subtraj_A,subtraj_B):
    
    """ fit only works now on a single frame (mdtraj returns xyz with shape (n_frames, atoms, xyz) 
         even for single frame trajs so hence the xyz[0]"""
    
    # load super imposer
    sup = SVDSuperimposer()

    # Set the coords, y will be rotated and translated on x
    x = subtraj_A.xyz[0]
    y = subtraj_B.xyz[0]
    sup.set(x, y)

    # Do the leastsquared fit
    sup.run()

    # Get the rms
    rms = sup.get_rms()

    # Get rotation (right multiplying!) and the translation
    rot, tran = sup.get_rotran()
    
    # now we have the instructions to rotate B on A
    return rot,tran,rms

def apply_superimposition(traj, rot, tran):
    
    # get xyz coordinates
    xyz = traj.xyz[0]
    
    # rotate subject on target
    new_xyz = dot(xyz, rot) + tran

    # replace coordinates of traj
    traj.xyz = new_xyz
    return traj

def update_topology(C):
    top = C.top
    # Merge two tops (with two chains or more) to a top of one chain 
    out = md.Topology()
    c = out.add_chain()
    for chain in top.chains:

        for residue in chain.residues:
            r = out.add_residue(residue.name, c, residue.resSeq, residue.segment_id)
            for atom in residue.atoms:
                out.add_atom(atom.name, atom.element, r, serial=atom.serial)
    #     for bond in top.bonds:
    #         a1, a2 = bond
    #         out.add_bond(a1, a2, type=bond.type, order=bond.order)
    out.create_standard_bonds() #rare manier om bonds te maken, maar werkt
    C.top = out 
    return C

def fit_B_on_A(A, B, selection_A, selection_B, delete_overlap=False):
    
    # create trajs containing only the selections
    subtraj_A = A.atom_slice(selection_A)
    subtraj_B = B.atom_slice(selection_B)

    # obtain instructions to rotate and translate B on A based on substraj structures
    rot, tran, rms = get_rot_and_trans(subtraj_A,subtraj_B)
    
    # do the superimposition of B on A and subsitute old with new xyz of B
    new_B = apply_superimposition(B, rot, tran)

    
    if delete_overlap:
        # remove selection from A and add new B
        selection_to_delete = selection_A 
        new_A = A.atom_slice([at.index for at in A.top.atoms if at.index not in selection_to_delete])

        # merge superimposed structure and fit (A and B*) and update topology such that they share a chain
        C = update_topology(new_A.stack(new_B))
    else:
         C = A.stack(new_B)
            
    return C, rms

In [ ]:
# Load H-NS dimers
loc_dimers = './data/dimer_pdbs/'
dimers = md.join([md.load(loc_dimers+f'run{i}.pdb') for i in range(0,16)])

In [ ]:
# create small piece of protein
traj = dimers
top = traj.topology
selection = top.select('resid 60 to 79')
print(traj)
traj = dimers.atom_slice(selection)
view  = nv.show_mdtraj(traj.center_coordinates())
view

In [ ]:
# split the small piece in 2 as a demo on how to "repair/graft" it to once piece again
traj_A = traj.atom_slice(traj.top.select('resid 0 to 11'))
traj_B = traj.atom_slice(traj.top.select('resid 10 to 20'))

# pick residues that overlap and that you will use for the fit
selection_A = traj_A.top.select('resid 10 11')
selection_B = traj_B.top.select('resid 0 1')

print(len(selection_A))
print(len(selection_B))

In [ ]:
C = None
# Fit B on A with the overlap atoms, here I used frame zero from piece A and frame 2 from piece B, you can of course change this easily
C, rms = fit_B_on_A(traj_A[3], traj_B[10], selection_A, selection_B, delete_overlap=True)
print(rms)
C.top